##### Fernando Amaral
##### CrossValidator

In [ ]:
# Tunning + Pipeline Com CrossValidation


In [1]:
#findspark para o notebook encontrar o Spark, deve ser instalado
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tunningcross").getOrCreate()

In [2]:
#inferschema determina que o schema seja descoberto pelo spark
iris = spark.read.csv("iris.csv", header=True, inferSchema=True, sep=",")
#divisão dos dados em treino e teste
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

In [3]:
#vetorização: nesta etapa criamos um atributos (coluna) que contém
#todas as variáveis independentes que queremos usar no modelo
from pyspark.ml.feature import VectorAssembler
vector = VectorAssembler(inputCols=["sepallength","sepalwidth","petallength",
                                    "petalwidth"],outputCol="independente" )

In [4]:
#Indexação da Classe: de categoria para número
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="class", outputCol="label")

In [5]:
#vamos criar uma RNA, os hiper parâmetros maxiter e layers vamos 
#utilizar no tunning 
from pyspark.ml.classification import MultilayerPerceptronClassifier
mlp = MultilayerPerceptronClassifier(maxIter=100, 
                                     layers=[4,5,4,3], featuresCol="independente", labelCol="label")

In [6]:
#pipeline: etapas que serão executadas no nosso processo
#usamos o nome dos objetos (variáveis) na sequência de execução
# vetorização, indexaçao e o modelo 
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vector, indexer, mlp])

In [9]:
#CrossValidator é a classe de tunning
#ParamGridBuilder usamos para contruir uma matriz com os hiper parâmetros a serem testados
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#precisamos de uma métrica para que CrossValidator avalie o melhor modelo
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#criamos o objeto  MulticlassClassificationEvaluator para avaliar a performance
performance = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")

# Construimos o grid com os hiper parametros que queremos testar
# por exemplo, mlp.maxIter,[10,100,1000], quer dizer que do objeto mlp, hiper parametros maxIter
# queremos testar 10,100 e 1000
grid = ParamGridBuilder().addGrid(mlp.maxIter,[10,100,1000]).addGrid(mlp.layers,[[4,5,4,3],[4,4,3]]).build()

# criamos o crossvalidotor com todos os parametros, isso ainda não executa, apenas cria a instância
crossval = CrossValidator(estimator=pipeline,estimatorParamMaps=grid,evaluator=performance,numFolds=3)

In [10]:
#execução, neste momento executa o pipeline a busca o melhor modelo
modelo = crossval.fit(irisTreino)

In [11]:
# faz a previsão usando o melhor modelo
previsao = modelo.transform(irisTeste)
# exibe a previsão, além de prediction cria rawprediction e probability
previsao.select("rawprediction","probability","prediction").show()

+--------------------+--------------------+----------+
|       rawprediction|         probability|prediction|
+--------------------+--------------------+----------+
|[-65.012837093793...|[2.56470034230802...|       2.0|
|[-65.012837093792...|[2.56470034230925...|       2.0|
|[-65.012837093792...|[2.56470034230889...|       2.0|
|[-65.012837093792...|[2.56470034230889...|       2.0|
|[-65.012837093792...|[2.56470034230860...|       2.0|
|[-65.012837093792...|[2.56470034230860...|       2.0|
|[-65.012837093793...|[2.56470034230831...|       2.0|
|[-65.012837093792...|[2.56470034230947...|       2.0|
|[-65.012837093792...|[2.56470034230925...|       2.0|
|[-65.012837093792...|[2.56470034230925...|       2.0|
|[-65.012837093792...|[2.56470034230925...|       2.0|
|[-65.012837093792...|[2.56470034230925...|       2.0|
|[-65.012837093792...|[2.56470034230918...|       2.0|
|[-7.1440309466339...|[1.21456141799657...|       1.0|
|[-65.012837093792...|[2.56470034230925...|       2.0|
|[-65.0128

In [12]:
# avaliamos a performance do melhor modelo usando os dados de teste
performance = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.9782608695652174
